In [3]:
import numpy as np
import sympy as sym
from numpy import cos, sin, sqrt, radians, degrees, pi, arcsin,arccos, tan
from numpy.linalg import norm
from scipy.optimize import fsolve

## Lambert's Problem

In [31]:
def Lambert(r1, r2, theta, t):
    """
    r1: initial position, in AU
    r2: final position, in AU
    theta: transfer angle, in degrees
    t: time of flight, in TU
    returns a, e
    """
    theta = radians(theta)
    
    # Calculating the chord and semiperimeter
    c = sqrt(r1**2 + r2**2 - 2*r1*r2*cos(theta))
    s = (r1 + r2 + c)/2
    betam = 2*arcsin(sqrt((s-c)/s))
    
    # Calculate the minimum flight time possible, which is the parabolic trajectory, tp
    tp = sqrt(2)/3 * (s**1.5 - np.sign(sin(theta))*(s - c)**1.5)
    
    # Calculate the minimum energy time, tm
    
    tm = sqrt(s**3/8) * (pi - betam + sin(betam))
    
    # Check if the given time of flight is greater or not
    if t < tp:
        return f'Time of flight not possible with a Lambert trajectory. Choose a time greater than {tp} TU'
    elif t > tp:
        # Create the function that solves for the time of flight
        def TOF(a):
            alpha0 = 2*arcsin(sqrt(s/(2*a)))
            beta0 = 2*arcsin(sqrt((s-c)/(2*a)))
            if theta < 180 or theta == 180:
                beta = beta0
            elif theta > 180:
                beta = - beta0
            
            if t < tm:
                alpha = alpha0
            elif t > tm:
                alpha = 2*pi - alpha0
                
            return t - a**1.5 *(alpha - beta - (sin(alpha) - sin(beta)))
        
        a = fsolve(TOF, 50)
        
        alpha0 = 2*arcsin(sqrt(s/(2*a)))
        beta0 = 2*arcsin(sqrt((s-c)/(2*a)))
        if theta < 180 or theta == 180:
            beta = beta0
        elif theta > 180:
            beta = - beta0

        if t < tm:
            alpha = alpha0
        elif t > tm:
            alpha = 2*pi - alpha0
        
        term = (4*(s - r1)*(s - r2))/c**2 * (sin((alpha + beta)/2))**2
        
        e = sqrt(1 - term)
        
        A = sqrt(1/(4*a)) * 1/tan(alpha/2)
        B = sqrt(1/(4*a)) * 1/tan(beta/2)
        
        u1 = np.array([1, 0])
        u2 = np.array([cos(theta), sin(theta)])
        
        theta_c = arcsin(sin(radians(theta))/c * r2)
        
        uc = np.array([-cos(theta_c), sin(theta_c)])
        
        v1 = (B + A)*uc + (B - A)*u1
        v2 = (B + A)*uc - (B - A)*u2
        
        return a, e, v1, v2

In [32]:
r1 = 0.3871
r2 = 39.5294
theta = 135
t = 100000 * 2*pi/365

[a, e, v1, v2] = Lambert(r1, r2, theta, t)

a = a[0]
e = e[0]

print(f'Semimajor axis: {np.round(a, 4)} AU \n')
print(f'Eccentricity: {np.round(e, 4)} \n')
print(f'Departure velocity: {np.round(v1, 4)} AU/TU \n')
print(f'Arrival velocity: {np.round(v2, 4)} AU/TU \n')

dv1 = sqrt(2/r1 - 1/a) - sqrt(1/r1)
dv2 = sqrt(1/r2) - sqrt(2/r2 - 1/a)

print(f'Delta v1: {np.round(dv1, 4)} AU/TU \n')
print(f'Delta v2: {np.round(dv2, 4)} AU/TU')

Semimajor axis: 44.4418 AU 

Eccentricity: 0.993 

Departure velocity: [0.1688 0.1183] AU/TU 

Arrival velocity: [-0.7288 -2.0486] AU/TU 

Delta v1: 0.6608 AU/TU 

Delta v2: -0.0086 AU/TU


## Hohmann Transfer

In [21]:
def Hohmann(r1, r2):
    # The semimajor axis for Hohmann Transfer
    a = (r1 + r2)/2
    
    dv1 = sqrt(2/r1 - 1/a) - sqrt(1/r1)
    dv2 = sqrt(1/r2) - sqrt(2/r2 - 1/a)
    
    return dv1, dv2

In [22]:
[dv1, dv2] = Hohmann(r1, r2)
print(f'Delta v1: {np.round(dv1, 4)} AU/TU \n')
print(f'Delta v2: {np.round(dv2, 4)} AU/TU')

Delta v1: 0.6432 AU/TU 

Delta v2: 0.1829 AU/TU
